In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
address = "https://raw.githubusercontent.com/Jaeik-Jeong/RLES/main/"

data_price = pd.read_csv(address+'price.csv', index_col=0)
data_train_csv = pd.DataFrame(data_price['Price'][:1000])
data_val_csv   = pd.DataFrame(data_price['Price'][1000:1500])
data_test_csv  = pd.DataFrame(data_price['Price'][1500:2000])

In [ ]:
# Data Preprocessing

Battery_Size = 1 #p.u.

max_price = max(data_price['Price'])

size_train0 = len(data_train_csv)
size_val0   = len(data_val_csv)
size_test0  = len(data_test_csv)

price_train = list(data_train_csv['Price']/max_price)
price_val = list(data_val_csv['Price']/max_price)
price_test = list(data_test_csv['Price']/max_price)

In [ ]:
# Agent

n_layers         = 2
in_size          = 2
hidden_size      = 64
out_size         = 1
T_horizon        = 128
learning_rate    = 0.001
K_epoch          = 3
gamma            = 0.99
lmbda            = 0.95
eps_clip         = 0.1
C_value          = 1
var              = 0.1**2

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.fc_s  = nn.Linear(in_size, hidden_size)
        self.rnn   = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.fc_pi = nn.Linear(hidden_size, out_size)
        self.fc_v  = nn.Linear(hidden_size, 1)

    def pi(self, x, hidden):
        x = F.relu(self.fc_s(x))
        x = x.view(1, -1, hidden_size)
        x, hidden = self.rnn(x, hidden)
        pi = self.fc_pi(x)
        pi = pi.view(-1, out_size)
        return pi, hidden

    def v(self, x, hidden):
        x = F.relu(self.fc_s(x))
        x = x.view(1, -1, hidden_size)
        x, hidden = self.rnn(x, hidden)
        v = self.fc_v(x)
        v = v.view(-1, 1)
        return v

In [ ]:
# Environment

E_max   = Battery_Size
P_max   = E_max
tdelta  = 0.5
soc_min = 0.1
soc_max = 0.9
a0 = -1.031; a1 = 35; a2 = 3.685; a3 = 0.2156; a4 = 0.1178; a5 = 0.3201
b0 = 0.1463; b1 = 30.27; b2 = 0.1037; b3 = 0.0584; b4 = 0.1747; b5 = 0.1288
c0 = 0.1063; c1 = 62.49; c2 = 0.0437; d0 = 0.0712; d1 = 61.4; d2 = 0.0288
N = 130*215*E_max/0.1
beta = 10/max_price

class Env():
    def __init__(self, data):
        self.data_imb = data
        self.state = []

    def reset(self):
        imb = self.data_imb[0]
        E = E_max/2
        state = [[imb, E]]
        self.state = state
        return state

    def step(self, action):
        imb = self.data_imb[len(self.state)]

        E = self.state[-1][-1]
        soc = E/E_max
        Voc = a0*np.exp(-a1*soc) + a2 + a3*soc - a4*soc**2 + a5*soc**3
        Rs  = b0*np.exp(-b1*soc) + b2 + b3*soc - b4*soc**2 + b5*soc**3
        Rts = c0*np.exp(-c1*soc) + c2
        Rtl = d0*np.exp(-d1*soc) + d2
        R   = Rs + Rts + Rtl

        I_cmax = 1000000*(E_max*soc_max - E)/N/(Voc*tdelta)
        I_dmax = 1000000*(E - E_max*soc_min)/N/(Voc*tdelta)
        p_cmax = N*(Voc*I_cmax + I_cmax**2*R)
        p_dmax = N*(Voc*I_dmax - I_dmax**2*R)

        P_cmax = p_cmax/1000000; P_dmax = p_dmax/1000000
        P_c = min(max(-action[0]*E_max, 0), P_max, P_cmax)
        P_d = min(max(action[0]*E_max,  0), P_max, P_dmax)
        p_c = 1000000*P_c/N; p_d = 1000000*P_d/N

        I_c = -(Voc - np.sqrt(Voc**2 + 4*R*p_c))/(2*R)
        I_d = (Voc - np.sqrt(Voc**2 - 4*R*p_d))/(2*R)
        if not np.isclose(p_c, 0):
            eff_c = (Voc*I_c)/p_c; eff_d = 1
            E_prime = E + eff_c*P_c*tdelta
            disp = -P_c
        elif not np.isclose(p_d, 0):
            eff_d = p_d/(Voc*I_d); eff_c = 1
            E_prime = E - (1/eff_d)*P_d*tdelta
            disp = P_d
        else:
            eff_c = 1; eff_d = 1
            E_prime = E
            disp = 0

        revenue = (imb*(P_d-P_c) - beta*(P_c+P_d))*tdelta

        next_state = self.state + [[imb, E_prime]]
        reward = revenue
        done = False
        info = [E, -P_c, P_d, revenue]

        self.state = next_state
        return next_state, reward, done, info

In [ ]:
# Environment 2

E_max   = Battery_Size
P_max   = E_max
tdelta  = 0.5
soc_min = 0.1
soc_max = 0.9
a0 = -1.031; a1 = 35; a2 = 3.685; a3 = 0.2156; a4 = 0.1178; a5 = 0.3201
b0 = 0.1463; b1 = 30.27; b2 = 0.1037; b3 = 0.0584; b4 = 0.1747; b5 = 0.1288
c0 = 0.1063; c1 = 62.49; c2 = 0.0437; d0 = 0.0712; d1 = 61.4; d2 = 0.0288
N = 130*215*E_max/0.1
beta = 1/max_price

class Env2():
    def __init__(self, data):
        self.data_imb = data
        self.state = []

    def reset(self):
        imb = self.data_imb[0]
        E = E_max/2
        state = [[imb, E]]
        self.state = state
        return state

    def step(self, action):
        imb = self.data_imb[len(self.state)]

        E = self.state[-1][-1]
        soc = E/E_max
        Voc = a0*np.exp(-a1*soc) + a2 + a3*soc - a4*soc**2 + a5*soc**3
        Rs  = b0*np.exp(-b1*soc) + b2 + b3*soc - b4*soc**2 + b5*soc**3
        Rts = c0*np.exp(-c1*soc) + c2
        Rtl = d0*np.exp(-d1*soc) + d2
        R   = Rs + Rts + Rtl

        I_cmax = 1000000*(E_max*soc_max - E)/N/(Voc*tdelta)
        I_dmax = 1000000*(E - E_max*soc_min)/N/(Voc*tdelta)
        p_cmax = N*(Voc*I_cmax + I_cmax**2*R)
        p_dmax = N*(Voc*I_dmax - I_dmax**2*R)

        P_cmax = p_cmax/1000000; P_dmax = p_dmax/1000000
        P_c = min(max(-action[0]*E_max, 0), P_max, P_cmax)
        P_d = min(max(action[0]*E_max,  0), P_max, P_dmax)
        p_c = 1000000*P_c/N; p_d = 1000000*P_d/N

        I_c = -(Voc - np.sqrt(Voc**2 + 4*R*p_c))/(2*R)
        I_d = (Voc - np.sqrt(Voc**2 - 4*R*p_d))/(2*R)
        if not np.isclose(p_c, 0):
            eff_c = (Voc*I_c)/p_c; eff_d = 1
            E_prime = E + eff_c*P_c*tdelta
            disp = -P_c
        elif not np.isclose(p_d, 0):
            eff_d = p_d/(Voc*I_d); eff_c = 1
            E_prime = E - (1/eff_d)*P_d*tdelta
            disp = P_d
        else:
            eff_c = 1; eff_d = 1
            E_prime = E
            disp = 0

        revenue = (imb*(P_d-P_c) - beta*(P_c+P_d))*tdelta

        next_state = self.state + [[imb, E_prime]]
        reward = (imb*(P_d-P_c) - beta*(P_c+P_d) - abs(P_c-max(-action[0]*E_max,0)) - abs(P_d-max(action[0]*E_max,0)))*tdelta
        done = False
        info = [E, -P_c, P_d, revenue]

        self.state = next_state
        return next_state, reward, done, info

In [ ]:
def train_net(model, batch, optimizer):
    o, H, a, r, o_prime, H_prime, done = [], [], [], [], [], [], []
    for transition in batch[0]:
        o.append(transition[0])
        a.append(transition[1])
        r.append([transition[2]])
        o_prime.append(transition[3])
        done.append([0]) if transition[4] else done.append([1])
    for transition in batch[1]:
        H.append(transition[0])
        H_prime.append(transition[1])

    o         = torch.tensor(o,dtype=torch.float)
    H         = (H[0][0].detach(), H[0][1].detach())
    a         = torch.tensor(a,dtype=torch.float)
    r         = torch.tensor(r,dtype=torch.float)
    o_prime   = torch.tensor(o_prime,dtype=torch.float)
    H_prime   = (H_prime[0][0].detach(), H_prime[0][1].detach())
    done      = torch.tensor(done)

    pdf_old = torch.distributions.MultivariateNormal(model.pi(o, H)[0], var*torch.eye(out_size))
    prob_old = torch.exp(pdf_old.log_prob(a)).view(len(a),1)
    prob_old = prob_old.detach()

    v_target = r + gamma * model.v(o_prime, H_prime) * done
    td = r + gamma * model.v(o_prime, H_prime) * done - model.v(o, H)
    td = td.detach().numpy()
    advantage = []
    A = 0.0
    for delta in td[::-1].flatten():
        A = delta + gamma*lmbda*A
        advantage.append([A])
    advantage.reverse()
    advantage = torch.tensor(advantage, dtype=torch.float)

    soc = o[:,1:]/E_max
    Voc = a0*np.exp(-a1*soc) + a2 + a3*soc - a4*soc**2 + a5*soc**3
    Rs  = b0*np.exp(-b1*soc) + b2 + b3*soc - b4*soc**2 + b5*soc**3
    Rts = c0*np.exp(-c1*soc) + c2
    Rtl = d0*np.exp(-d1*soc) + d2
    R   = Rs + Rts + Rtl

    I_cmax = 1000000*(E_max*soc_max - o[:,1:])/N/(Voc*tdelta)
    I_dmax = 1000000*(o[:,1:] - E_max*soc_min)/N/(Voc*tdelta)
    p_cmax = N*(Voc*I_cmax + I_cmax**2*R)
    p_dmax = N*(Voc*I_dmax - I_dmax**2*R)

    P_cmax = p_cmax/1000000; P_dmax = p_dmax/1000000

    for i in range(K_epoch):
        loss_sup = F.mse_loss(torch.maximum(-P_cmax-model.pi(o, H)[0],torch.tensor(0.0)) + torch.minimum(P_dmax-model.pi(o, H)[0],torch.tensor(0.0)), torch.tensor([[0.0] for _ in range(len(a))]))

        pdf = torch.distributions.MultivariateNormal(model.pi(o, H)[0], var*torch.eye(out_size))
        prob = torch.exp(pdf.log_prob(a)).view(len(a),1)
        ratio = torch.exp(torch.log(prob) - torch.log(prob_old))  # a/b == exp(log(a)-log(b))

        loss_actor = torch.min(ratio * advantage, torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage)
        loss_critic = F.mse_loss(model.v(o, H), v_target.detach())
        loss = -(loss_actor - C_value*loss_critic)

        optimizer.zero_grad()
        loss.mean().backward(retain_graph=True)
        optimizer.step()

In [ ]:
def train_net2(model, batch, optimizer):
    o, H, a, r, o_prime, H_prime, done = [], [], [], [], [], [], []
    for transition in batch[0]:
        o.append(transition[0])
        a.append(transition[1])
        r.append([transition[2]])
        o_prime.append(transition[3])
        done.append([0]) if transition[4] else done.append([1])
    for transition in batch[1]:
        H.append(transition[0])
        H_prime.append(transition[1])

    o         = torch.tensor(o,dtype=torch.float)
    H         = (H[0][0].detach(), H[0][1].detach())
    a         = torch.tensor(a,dtype=torch.float)
    r         = torch.tensor(r,dtype=torch.float)
    o_prime   = torch.tensor(o_prime,dtype=torch.float)
    H_prime   = (H_prime[0][0].detach(), H_prime[0][1].detach())
    done      = torch.tensor(done)

    pdf_old = torch.distributions.MultivariateNormal(model.pi(o, H)[0], var*torch.eye(out_size))
    prob_old = torch.exp(pdf_old.log_prob(a)).view(len(a),1)
    prob_old = prob_old.detach()

    v_target = r + gamma * model.v(o_prime, H_prime) * done
    td = r + gamma * model.v(o_prime, H_prime) * done - model.v(o, H)
    td = td.detach().numpy()
    advantage = []
    A = 0.0
    for delta in td[::-1].flatten():
        A = delta + gamma*lmbda*A
        advantage.append([A])
    advantage.reverse()
    advantage = torch.tensor(advantage, dtype=torch.float)

    soc = o[:,1:]/E_max
    Voc = a0*np.exp(-a1*soc) + a2 + a3*soc - a4*soc**2 + a5*soc**3
    Rs  = b0*np.exp(-b1*soc) + b2 + b3*soc - b4*soc**2 + b5*soc**3
    Rts = c0*np.exp(-c1*soc) + c2
    Rtl = d0*np.exp(-d1*soc) + d2
    R   = Rs + Rts + Rtl

    I_cmax = 1000000*(E_max*soc_max - o[:,1:])/N/(Voc*tdelta)
    I_dmax = 1000000*(o[:,1:] - E_max*soc_min)/N/(Voc*tdelta)
    p_cmax = N*(Voc*I_cmax + I_cmax**2*R)
    p_dmax = N*(Voc*I_dmax - I_dmax**2*R)

    P_cmax = p_cmax/1000000; P_dmax = p_dmax/1000000

    for i in range(K_epoch):
        loss_sup = F.mse_loss(torch.maximum(-P_cmax-model.pi(o, H)[0],torch.tensor(0.0)) + torch.minimum(P_dmax-model.pi(o, H)[0],torch.tensor(0.0)), torch.tensor([[0.0] for _ in range(len(a))]))

        pdf = torch.distributions.MultivariateNormal(model.pi(o, H)[0], var*torch.eye(out_size))
        prob = torch.exp(pdf.log_prob(a)).view(len(a),1)
        ratio = torch.exp(torch.log(prob) - torch.log(prob_old))  # a/b == exp(log(a)-log(b))

        loss_actor = torch.min(ratio * advantage, torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage)
        loss_critic = F.mse_loss(model.v(o, H), v_target.detach())
        loss = -(loss_actor - loss_sup - C_value*loss_critic)

        optimizer.zero_grad()
        loss.mean().backward(retain_graph=True)
        optimizer.step()

In [ ]:
model1 = LSTM()
env_train = Env(price_train)
env_val   = Env(price_val)
env_test  = Env(price_test)
bat_train, bat_val, bat_test = [], [], [] # SoC
cha_train, cha_val, cha_test = [], [], [] # Charging
dis_train, dis_val, dis_test = [], [], [] # Discharging
rev_train, rev_val, rev_test = [], [], [] # Revenue
total_episode = 10
max_iteration = int(len(data_train_csv)/T_horizon)
print_interval = 1

optimizer = torch.optim.Adam(model1.parameters(), lr=learning_rate)
REV_train_list = []; REV_val_list = []; REV_test_list = []
for n_epi in range(total_episode):
    bat_train += [[]]; bat_val += [[]]; bat_test += [[]]
    cha_train += [[]]; cha_val += [[]]; cha_test += [[]]
    dis_train += [[]]; dis_val += [[]]; dis_test += [[]]
    rev_train += [[]]; rev_val += [[]]; rev_test += [[]]

    state = env_train.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for i in range(max_iteration):
        batch = [[],[]]
        for t in range(T_horizon):
            pi_out, next_history = model1.pi(torch.tensor(state[-1], dtype=torch.float), history)
            action = np.random.multivariate_normal(pi_out.detach().numpy()[0], var*np.identity(out_size), 1)[0].tolist()
            next_state, reward, done, info = env_train.step(action)

            batch[0].append((state[-1], action, reward, next_state[-1], done))
            batch[1].append((history, next_history))
            state = next_state[:]
            history = next_history

            bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
            bat_train[n_epi] += [bat]
            cha_train[n_epi] += [cha]
            dis_train[n_epi] += [dis]
            rev_train[n_epi] += [revenue]
            if done:
                break

        if n_epi != 0:
            train_net(model1, batch, optimizer)
        if done:
            break

    state = env_val.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for k in range(len(env_val.data_imb)-1):
        pi_out, next_history = model1.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_val.step(action)

        state = next_state[:]
        history = next_history

        bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
        bat_val[n_epi] += [bat]
        cha_val[n_epi] += [cha]
        dis_val[n_epi] += [dis]
        rev_val[n_epi] += [revenue]

    state = env_test.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for l in range(len(env_test.data_imb)-1):
        pi_out, next_history = model1.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_test.step(action)

        state = next_state[:]
        history = next_history

        bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
        bat_test[n_epi] += [bat]
        cha_test[n_epi] += [cha]
        dis_test[n_epi] += [dis]
        rev_test[n_epi] += [revenue]

    if (n_epi+1)%print_interval == 0:
        REV_train = round(max_price*100*np.mean(rev_train[n_epi]),3); REV_train_list += [REV_train]
        REV_val   = round(max_price*100*np.mean(rev_val[n_epi]),3); REV_val_list += [REV_val]
        REV_test  = round(max_price*100*np.mean(rev_test[n_epi]),3); REV_test_list += [REV_test]

        print("episode: {}".format(n_epi+1))
        print("REV_train: ${}".format(REV_train).ljust(25), end="")
        print("REV_val: ${}".format(REV_val).ljust(25), end="")
        print("REV_test: ${}".format(REV_test).ljust(25))
        print("------------------------------------------------------------------------------------------")

    if REV_val_list[-1] == np.max(REV_val_list):
        torch.save(model1.state_dict(), 'MODEL1.pt')

In [ ]:
model1_test = LSTM()
model1_test.load_state_dict(torch.load('MODEL1.pt'))

bat_val = []; bat_test = []
cha_val = []; cha_test = []
dis_val = []; dis_test = []
rev_val = []; rev_test = []

state = env_val.reset()
history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
for k in range(len(env_val.data_imb)-1):
    pi_out, next_history = model1_test.pi(torch.tensor(state[-1], dtype=torch.float), history)
    action = pi_out[0].tolist()
    next_state, reward, done, info = env_val.step(action)

    state = next_state[:]
    history = next_history

    bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
    bat_val += [bat]
    cha_val += [cha]
    dis_val += [dis]
    rev_val += [revenue]

state = env_test.reset()
history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
for l in range(len(env_test.data_imb)-1):
    pi_out, next_history = model1_test.pi(torch.tensor(state[-1], dtype=torch.float), history)
    action = pi_out[0].tolist()
    next_state, reward, done, info = env_test.step(action)

    state = next_state[:]
    history = next_history

    bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
    bat_test += [bat]
    cha_test += [cha]
    dis_test += [dis]
    rev_test += [revenue]

REV_val   = round(max_price*100*np.mean(rev_val),3)
REV_test  = round(max_price*100*np.mean(rev_test),3)
print("REV_val: ${}".format(REV_val).ljust(25), end="")
print("REV_test: ${}".format(REV_test).ljust(25))
print("------------------------------------------------------------------------------------------")

plt.plot(bat_test)

In [ ]:
model2 = LSTM()
env_train = Env2(price_train)
env_val   = Env2(price_val)
env_test  = Env2(price_test)
bat_train, bat_val, bat_test = [], [], [] # SoC
cha_train, cha_val, cha_test = [], [], [] # Charging
dis_train, dis_val, dis_test = [], [], [] # Discharging
rev_train, rev_val, rev_test = [], [], [] # Revenue
total_episode = 100
max_iteration = int(len(data_train_csv)/T_horizon)
print_interval = 1

optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)
REV_train_list = []; REV_val_list = []; REV_test_list = []
for n_epi in range(total_episode):
    bat_train += [[]]; bat_val += [[]]; bat_test += [[]]
    cha_train += [[]]; cha_val += [[]]; cha_test += [[]]
    dis_train += [[]]; dis_val += [[]]; dis_test += [[]]
    rev_train += [[]]; rev_val += [[]]; rev_test += [[]]

    state = env_train.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for i in range(max_iteration):
        batch = [[],[]]
        for t in range(T_horizon):
            pi_out, next_history = model2.pi(torch.tensor(state[-1], dtype=torch.float), history)
            action = np.random.multivariate_normal(pi_out.detach().numpy()[0], var*np.identity(out_size), 1)[0].tolist()
            next_state, reward, done, info = env_train.step(action)

            batch[0].append((state[-1], action, reward, next_state[-1], done))
            batch[1].append((history, next_history))
            state = next_state[:]
            history = next_history

            bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
            bat_train[n_epi] += [bat]
            cha_train[n_epi] += [cha]
            dis_train[n_epi] += [dis]
            rev_train[n_epi] += [revenue]
            if done:
                break

        if n_epi != 0:
            train_net(model2, batch, optimizer)
        if done:
            break

    state = env_val.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for k in range(len(env_val.data_imb)-1):
        pi_out, next_history = model2.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_val.step(action)

        state = next_state[:]
        history = next_history

        bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
        bat_val[n_epi] += [bat]
        cha_val[n_epi] += [cha]
        dis_val[n_epi] += [dis]
        rev_val[n_epi] += [revenue]

    state = env_test.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for l in range(len(env_test.data_imb)-1):
        pi_out, next_history = model2.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_test.step(action)

        state = next_state[:]
        history = next_history

        bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
        bat_test[n_epi] += [bat]
        cha_test[n_epi] += [cha]
        dis_test[n_epi] += [dis]
        rev_test[n_epi] += [revenue]

    if (n_epi+1)%print_interval == 0:
        REV_train = round(max_price*100*np.mean(rev_train[n_epi]),3); REV_train_list += [REV_train]
        REV_val   = round(max_price*100*np.mean(rev_val[n_epi]),3); REV_val_list += [REV_val]
        REV_test  = round(max_price*100*np.mean(rev_test[n_epi]),3); REV_test_list += [REV_test]

        print("episode: {}".format(n_epi+1))
        print("REV_train: ${}".format(REV_train).ljust(25), end="")
        print("REV_val: ${}".format(REV_val).ljust(25), end="")
        print("REV_test: ${}".format(REV_test).ljust(25))
        print("------------------------------------------------------------------------------------------")

    if REV_val_list[-1] == np.max(REV_val_list):
        torch.save(model2.state_dict(), 'MODEL2.pt')

In [ ]:
model2_test = LSTM()
model2_test.load_state_dict(torch.load('MODEL2.pt'))

bat_val = []; bat_test = []
cha_val = []; cha_test = []
dis_val = []; dis_test = []
rev_val = []; rev_test = []

state = env_val.reset()
history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
for k in range(len(env_val.data_imb)-1):
    pi_out, next_history = model2_test.pi(torch.tensor(state[-1], dtype=torch.float), history)
    action = pi_out[0].tolist()
    next_state, reward, done, info = env_val.step(action)

    state = next_state[:]
    history = next_history

    bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
    bat_val += [bat]
    cha_val += [cha]
    dis_val += [dis]
    rev_val += [revenue]

state = env_test.reset()
history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
for l in range(len(env_test.data_imb)-1):
    pi_out, next_history = model2_test.pi(torch.tensor(state[-1], dtype=torch.float), history)
    action = pi_out[0].tolist()
    next_state, reward, done, info = env_test.step(action)

    state = next_state[:]
    history = next_history

    bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
    bat_test += [bat]
    cha_test += [cha]
    dis_test += [dis]
    rev_test += [revenue]

REV_val   = round(max_price*100*np.mean(rev_val),3)
REV_test  = round(max_price*100*np.mean(rev_test),3)
print("REV_val: ${}".format(REV_val).ljust(25), end="")
print("REV_test: ${}".format(REV_test).ljust(25))
print("------------------------------------------------------------------------------------------")

plt.plot(bat_test)

In [ ]:
model3 = LSTM()
env_train = Env(price_train)
env_val   = Env(price_val)
env_test  = Env(price_test)
bat_train, bat_val, bat_test = [], [], [] # SoC
cha_train, cha_val, cha_test = [], [], [] # Charging
dis_train, dis_val, dis_test = [], [], [] # Discharging
rev_train, rev_val, rev_test = [], [], [] # Revenue
total_episode = 100
max_iteration = int(len(data_train_csv)/T_horizon)
print_interval = 1

optimizer = torch.optim.Adam(model3.parameters(), lr=learning_rate)
REV_train_list = []; REV_val_list = []; REV_test_list = []
for n_epi in range(total_episode):
    bat_train += [[]]; bat_val += [[]]; bat_test += [[]]
    cha_train += [[]]; cha_val += [[]]; cha_test += [[]]
    dis_train += [[]]; dis_val += [[]]; dis_test += [[]]
    rev_train += [[]]; rev_val += [[]]; rev_test += [[]]

    state = env_train.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for i in range(max_iteration):
        batch = [[],[]]
        for t in range(T_horizon):
            pi_out, next_history = model3.pi(torch.tensor(state[-1], dtype=torch.float), history)
            action = np.random.multivariate_normal(pi_out.detach().numpy()[0], var*np.identity(out_size), 1)[0].tolist()
            next_state, reward, done, info = env_train.step(action)

            batch[0].append((state[-1], action, reward, next_state[-1], done))
            batch[1].append((history, next_history))
            state = next_state[:]
            history = next_history

            bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
            bat_train[n_epi] += [bat]
            cha_train[n_epi] += [cha]
            dis_train[n_epi] += [dis]
            rev_train[n_epi] += [revenue]
            if done:
                break

        if n_epi != 0:
            train_net2(model3, batch, optimizer)
        if done:
            break

    state = env_val.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for k in range(len(env_val.data_imb)-1):
        pi_out, next_history = model3.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_val.step(action)

        state = next_state[:]
        history = next_history

        bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
        bat_val[n_epi] += [bat]
        cha_val[n_epi] += [cha]
        dis_val[n_epi] += [dis]
        rev_val[n_epi] += [revenue]

    state = env_test.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for l in range(len(env_test.data_imb)-1):
        pi_out, next_history = model2.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_test.step(action)

        state = next_state[:]
        history = next_history

        bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
        bat_test[n_epi] += [bat]
        cha_test[n_epi] += [cha]
        dis_test[n_epi] += [dis]
        rev_test[n_epi] += [revenue]

    if (n_epi+1)%print_interval == 0:
        REV_train = round(max_price*100*np.mean(rev_train[n_epi]),3); REV_train_list += [REV_train]
        REV_val   = round(max_price*100*np.mean(rev_val[n_epi]),3); REV_val_list += [REV_val]
        REV_test  = round(max_price*100*np.mean(rev_test[n_epi]),3); REV_test_list += [REV_test]

        print("episode: {}".format(n_epi+1))
        print("REV_train: ${}".format(REV_train).ljust(25), end="")
        print("REV_val: ${}".format(REV_val).ljust(25), end="")
        print("REV_test: ${}".format(REV_test).ljust(25))
        print("------------------------------------------------------------------------------------------")

    if REV_val_list[-1] == np.max(REV_val_list):
        torch.save(model3.state_dict(), 'MODEL3.pt')

In [ ]:
model3_test = LSTM()
model3_test.load_state_dict(torch.load('MODEL3.pt'))

bat_val = []; bat_test = []
cha_val = []; cha_test = []
dis_val = []; dis_test = []
rev_val = []; rev_test = []

state = env_val.reset()
history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
for k in range(len(env_val.data_imb)-1):
    pi_out, next_history = model3_test.pi(torch.tensor(state[-1], dtype=torch.float), history)
    action = pi_out[0].tolist()
    next_state, reward, done, info = env_val.step(action)

    state = next_state[:]
    history = next_history

    bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
    bat_val += [bat]
    cha_val += [cha]
    dis_val += [dis]
    rev_val += [revenue]

state = env_test.reset()
history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
for l in range(len(env_test.data_imb)-1):
    pi_out, next_history = model3_test.pi(torch.tensor(state[-1], dtype=torch.float), history)
    action = pi_out[0].tolist()
    next_state, reward, done, info = env_test.step(action)

    state = next_state[:]
    history = next_history

    bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
    bat_test += [bat]
    cha_test += [cha]
    dis_test += [dis]
    rev_test += [revenue]

REV_val   = round(max_price*100*np.mean(rev_val),3)
REV_test  = round(max_price*100*np.mean(rev_test),3)
print("REV_val: ${}".format(REV_val).ljust(25), end="")
print("REV_test: ${}".format(REV_test).ljust(25))
print("------------------------------------------------------------------------------------------")

plt.plot(bat_test)

In [ ]:
# def train_net2(model, batch, optimizer):
#     o, H, a, r, o_prime, H_prime, done = [], [], [], [], [], [], []
#     for transition in batch[0]:
#         o.append(transition[0])
#         a.append(transition[1])
#         r.append([transition[2]])
#         o_prime.append(transition[3])
#         done.append([0]) if transition[4] else done.append([1])
#     for transition in batch[1]:
#         H.append(transition[0])
#         H_prime.append(transition[1])

#     o         = torch.tensor(o,dtype=torch.float)
#     H         = (H[0][0].detach(), H[0][1].detach())
#     a         = torch.tensor(a,dtype=torch.float)
#     r         = torch.tensor(r,dtype=torch.float)
#     o_prime   = torch.tensor(o_prime,dtype=torch.float)
#     H_prime   = (H_prime[0][0].detach(), H_prime[0][1].detach())
#     done      = torch.tensor(done)

#     tanh = nn.Tanh()
#     pdf_old = torch.distributions.MultivariateNormal(tanh(model.pi(o, H)[0]), var*torch.eye(out_size))
#     prob_old = torch.exp(pdf_old.log_prob(a)).view(len(a),1)
#     prob_old = prob_old.detach()

#     v_target = r + gamma * model.v(o_prime, H_prime) * done
#     td = r + gamma * model.v(o_prime, H_prime) * done - model.v(o, H)
#     td = td.detach().numpy()
#     advantage = []
#     A = 0.0
#     for delta in td[::-1].flatten():
#         A = delta + gamma*lmbda*A
#         advantage.append([A])
#     advantage.reverse()
#     advantage = torch.tensor(advantage, dtype=torch.float)

#     for i in range(K_epoch):
#         pdf = torch.distributions.MultivariateNormal(tanh(model.pi(o, H)[0]), var*torch.eye(out_size))
#         prob = torch.exp(pdf.log_prob(a)).view(len(a),1)
#         ratio = torch.exp(torch.log(prob) - torch.log(prob_old))  # a/b == exp(log(a)-log(b))

#         loss_actor = torch.min(ratio * advantage, torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage)
#         loss_critic = F.mse_loss(model.v(o, H), v_target.detach())
#         loss = -(loss_actor - C_value*loss_critic)

#         optimizer.zero_grad()
#         loss.mean().backward(retain_graph=True)
#         optimizer.step()

In [ ]:
# def train_net2(model, batch, optimizer):
#     o, H, a, r, o_prime, H_prime, done = [], [], [], [], [], [], []
#     for transition in batch[0]:
#         o.append(transition[0])
#         a.append(transition[1])
#         r.append([transition[2]])
#         o_prime.append(transition[3])
#         done.append([0]) if transition[4] else done.append([1])
#     for transition in batch[1]:
#         H.append(transition[0])
#         H_prime.append(transition[1])

#     o         = torch.tensor(o,dtype=torch.float)
#     H         = (H[0][0].detach(), H[0][1].detach())
#     a         = torch.tensor(a,dtype=torch.float)
#     r         = torch.tensor(r,dtype=torch.float)
#     o_prime   = torch.tensor(o_prime,dtype=torch.float)
#     H_prime   = (H_prime[0][0].detach(), H_prime[0][1].detach())
#     done      = torch.tensor(done)

#     soc = o[:,1:]/E_max
#     Voc = a0*np.exp(-a1*soc) + a2 + a3*soc - a4*soc**2 + a5*soc**3
#     Rs  = b0*np.exp(-b1*soc) + b2 + b3*soc - b4*soc**2 + b5*soc**3
#     Rts = c0*np.exp(-c1*soc) + c2
#     Rtl = d0*np.exp(-d1*soc) + d2
#     R   = Rs + Rts + Rtl

#     I_cmax = 1000000*(E_max*soc_max - o[:,1:])/N/(Voc*tdelta)
#     I_dmax = 1000000*(o[:,1:] - E_max*soc_min)/N/(Voc*tdelta)
#     p_cmax = N*(Voc*I_cmax + I_cmax**2*R)
#     p_dmax = N*(Voc*I_dmax - I_dmax**2*R)

#     P_cmax = p_cmax/1000000; P_dmax = p_dmax/1000000

#     tanh = nn.Tanh()
#     pdf_old = torch.distributions.MultivariateNormal(tanh(model.pi(o, H)[0])*(P_cmax+P_dmax)/2+(P_dmax-P_cmax)/2, var*torch.eye(out_size))
#     prob_old = torch.exp(pdf_old.log_prob(a)).view(len(a),1)
#     prob_old = prob_old.detach()

#     v_target = r + gamma * model.v(o_prime, H_prime) * done
#     td = r + gamma * model.v(o_prime, H_prime) * done - model.v(o, H)
#     td = td.detach().numpy()
#     advantage = []
#     A = 0.0
#     for delta in td[::-1].flatten():
#         A = delta + gamma*lmbda*A
#         advantage.append([A])
#     advantage.reverse()
#     advantage = torch.tensor(advantage, dtype=torch.float)

#     for i in range(K_epoch):
#         loss_sup = F.mse_loss(torch.maximum(-P_cmax-model.pi(o, H)[0],torch.tensor(0.0)) + torch.minimum(P_dmax-model.pi(o, H)[0],torch.tensor(0.0)), torch.tensor([[0.0] for _ in range(len(a))]))

#         pdf = torch.distributions.MultivariateNormal(tanh(model.pi(o, H)[0])*(P_cmax+P_dmax)/2+(P_dmax-P_cmax)/2, var*torch.eye(out_size))
#         prob = torch.exp(pdf.log_prob(a)).view(len(a),1)
#         ratio = torch.exp(torch.log(prob) - torch.log(prob_old))  # a/b == exp(log(a)-log(b))

#         loss_actor = torch.min(ratio * advantage, torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage)
#         loss_critic = F.mse_loss(model.v(o, H), v_target.detach())
#         loss = -(loss_actor - C_value*loss_critic)

#         optimizer.zero_grad()
#         loss.mean().backward(retain_graph=True)
#         optimizer.step()

#     return loss_sup

In [ ]:
# model2 = LSTM()
# env_train = Env(price_train)
# env_val   = Env(price_val)
# env_test  = Env(price_test)
# bat_train, bat_val, bat_test = [], [], [] # Action
# cha_train, cha_val, cha_test = [], [], [] # Charging
# dis_train, dis_val, dis_test = [], [], [] # Discharging
# rev_train, rev_val, rev_test = [], [], [] # Revenue
# total_episode = 100
# max_iteration = int(len(data_train_csv)/T_horizon)
# print_interval = 1

# optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)
# for n_epi in range(total_episode):
#     loss_sup_list = [0]
#     bat_train += [[]]; bat_val += [[]]; bat_test += [[]]
#     cha_train += [[]]; cha_val += [[]]; cha_test += [[]]
#     dis_train += [[]]; dis_val += [[]]; dis_test += [[]]
#     rev_train += [[]]; rev_val += [[]]; rev_test += [[]]

#     state = env_train.reset()
#     history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
#     for i in range(max_iteration):
#         batch = [[],[]]
#         for t in range(T_horizon):
#             pi_out, next_history = model2.pi(torch.tensor(state[-1], dtype=torch.float), history)
#             action = np.random.multivariate_normal(pi_out.detach().numpy()[0], var*np.identity(out_size), 1)[0].tolist()
#             next_state, reward, done, info = env_train.step(action)

#             batch[0].append((state[-1], action, reward, next_state[-1], done))
#             batch[1].append((history, next_history))
#             state = next_state[:]
#             history = next_history

#             bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
#             bat_train[n_epi] += [bat]
#             cha_train[n_epi] += [cha]
#             dis_train[n_epi] += [dis]
#             rev_train[n_epi] += [revenue]
#             if done:
#                 break

#         if n_epi != 0:
#             loss_sup = train_net2(model2, batch, optimizer)
#             loss_sup_list += [float(loss_sup)]
#         if done:
#             break

#     state = env_val.reset()
#     history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
#     for k in range(len(env_val.data_imb)-1):
#         pi_out, next_history = model2.pi(torch.tensor(state[-1], dtype=torch.float), history)
#         action = pi_out[0].tolist()
#         next_state, reward, done, info = env_val.step(action)

#         state = next_state[:]
#         history = next_history

#         bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
#         bat_val[n_epi] += [bat]
#         cha_val[n_epi] += [cha]
#         dis_val[n_epi] += [dis]
#         rev_val[n_epi] += [revenue]

#     state = env_test.reset()
#     history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
#     for l in range(len(env_test.data_imb)-1):
#         pi_out, next_history = model2.pi(torch.tensor(state[-1], dtype=torch.float), history)
#         action = pi_out[0].tolist()
#         next_state, reward, done, info = env_test.step(action)

#         state = next_state[:]
#         history = next_history

#         bat = info[0]; cha = info[1]; dis = info[2]; revenue = info[3]
#         bat_test[n_epi] += [bat]
#         cha_test[n_epi] += [cha]
#         dis_test[n_epi] += [dis]
#         rev_test[n_epi] += [revenue]

#     if (n_epi+1)%print_interval == 0:
#         REV_train = round(max_price*100*np.mean(rev_train[n_epi]),3)
#         REV_val   = round(max_price*100*np.mean(rev_val[n_epi]),3)
#         REV_test  = round(max_price*100*np.mean(rev_test[n_epi]),3)

#         print("episode: {}".format(n_epi+1))
#         print("loss_sup: {}".format(round(np.mean(loss_sup_list),8)).ljust(25), end="")
#         print("REV_train: ${}".format(REV_train).ljust(25), end="")
#         print("REV_val: ${}".format(REV_val).ljust(25), end="")
#         print("REV_test: ${}".format(REV_test).ljust(25))
#         print("------------------------------------------------------------------------------------------")